# 서울열린데이터광장 재개발(주택정비형) 현황 지오코딩 & 공간 시각화

목표:
1. CSV (`서울열린데이터광장_재개발(주택정비형).csv`) 로드
2. `대표지번` 기반 주소 정리 & 지오코딩 (캐시 사용, 실패 재시도)
3. 서울시 행정구역 경계 Shapefile 로드 (사용자 제공 경로)
4. 포인트 GeoDataFrame 생성 후 경계와 공간조인 / Overlap
5. Folium 기반 인터랙티브 지도 + (선택) GeoPandas 정적 시각화

> Nominatim 사용 시 과다 요청 제한(1초 이상 간격 권장) → 캐시 필수. 영리 목적 사용 제한 유의.

---
필요 패키지: `pandas`, `geopy`, `geopandas`, `shapely`, `folium`, (선택) `tqdm`, `contextily`(배경)

아래 셀부터 순서대로 실행하세요.

In [198]:
from __future__ import annotations
"""
서울열린데이터광장 재개발(주택정비형) 데이터 지오코딩 & 공간 분석 파이프라인 (행정동 기반 버전)
- 시도(SIDO) 컬럼 사용 제거, 행정동(ADM_DR_NM / ADM_NM 등) 직접 매칭으로 서울 데이터 필터
- 행정동 리스트(SEOUL_DONG_NAMES) 교집합 비율로 가장 적합한 행정동 컬럼 자동 선택
- 자치구(SI/GU) 컬럼 탐색 / 파생 후 dissolve 로 25개 구 경계 생성 시도
- 단계별 디버그 / 실행시간 측정 유틸 포함
"""

import pandas as pd, json, re, os, time, contextlib
import geopandas as gpd
from shapely.geometry import Point
import folium
from folium.plugins import MarkerCluster
from pathlib import Path
from typing import List, Dict, Tuple, Optional
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter
from IPython.display import display  # 노트북 내 즉시 시각화용

# ===================== 사용자 설정 =====================
CSV_PATH = Path(r'C:/Users/Woojin Cho/Documents/GitHub/Urban-Refurbishing-Project_DATA-SET/outputs/analysis/서울열린데이터광장_재개발(주택정비형).csv')
OVERRIDE_SHAPE_PATH: Optional[Path] = None  # 예: Path(r'D:/gis/seoul/BND_ADM_DONG_PG.shp')
GEOCODE_DELAY_SEC = 1.0
USER_AGENT = 'seoul-redev-geocode'
CITY = '서울특별시'  # 주소 정규화 용도로만 사용 (시도 필터 X)

DEBUG = True  # 전역 디버그 스위치
PROGRESS_INTERVAL = 50  # 지오코딩 진행 로그 간격

OUT_DIR = CSV_PATH.parent
CACHE_PATH = OUT_DIR / 'geocode_cache.json'
MAP_BASIC = OUT_DIR / 'seoul_redev_map.html'
MAP_CHORO = OUT_DIR / 'seoul_redev_choropleth.html'
MAP_CLUSTER = OUT_DIR / 'seoul_redev_cluster.html'

# HTML 저장 여부 플래그 (False면 노트북 내 표시만 수행)
SAVE_HTML = False  # 필요시 True로 변경하면 기존처럼 파일도 생성

# 서울 행정동 이름 (필터 핵심)
SEOUL_DONG_NAMES = {
    '사직동','삼청동','부암동','평창동','무악동','교남동','가회동','종로1·2·3·4가동','종로5·6가동','이화동','창신1동','창신2동','창신3동','숭인1동','숭인2동','청운효자동','혜화동',
    '소공동','회현동','명동','필동','장충동','광희동','을지로동','신당5동','황학동','중림동','신당동','다산동','약수동','청구동','동화동',
    '후암동','용산2가동','남영동','원효로2동','효창동','용문동','이촌1동','이촌2동','이태원1동','이태원2동','서빙고동','보광동','청파동','원효로1동','한강로동','한남동',
    '왕십리2동','마장동','사근동','행당1동','행당2동','응봉동','금호1가동','금호4가동','성수1가1동','성수1가2동','성수2가1동','성수2가3동','송정동','용답동','왕십리도선동','금호2·3가동',
    '옥수동','화양동','군자동','중곡1동','중곡2동','중곡3동','중곡4동','능동','구의1동','구의2동','구의3동','광장동','자양1동','자양2동','자양3동','자양4동',
    '회기동','휘경1동','휘경2동','청량리동','용신동','제기동','전농1동','전농2동','답십리2동','장안1동','장안2동','이문1동','이문2동','답십리1동','면목2동','면목4동','면목5동','면목7동','상봉1동','상봉2동','중화1동','중화2동','묵1동','묵2동','망우3동','신내1동','신내2동','면목본동','면목3·8동','망우본동',
    '돈암1동','돈암2동','안암동','보문동','정릉1동','정릉2동','정릉3동','정릉4동','길음1동','길음2동','월곡1동','월곡2동','장위1동','장위2동','장위3동','성북동','삼선동','동선동','종암동','석관동',
    '번1동','번2동','번3동','수유1동','수유2동','수유3동','삼양동','미아동','송중동','송천동','삼각산동','우이동','인수동','쌍문1동','쌍문2동','쌍문3동','쌍문4동','방학1동','방학2동','방학3동','창1동','창2동','창3동','창4동','창5동','도봉1동','도봉2동',
    '월계1동','월계2동','월계3동','공릉2동','하계1동','하계2동','중계본동','중계1동','중계4동','상계1동','상계2동','상계5동','상계8동','상계9동','상계10동','상계3·4동','상계6·7동','중계2·3동','공릉1동',
    '녹번동','불광1동','갈현1동','갈현2동','구산동','대조동','응암1동','응암2동','신사1동','신사2동','증산동','수색동','진관동','불광2동','응암3동','역촌동',
    '천연동','홍제1동','홍제3동','홍제2동','홍은1동','홍은2동','남가좌1동','남가좌2동','북가좌1동','북가좌2동','충현동','북아현동','신촌동','연희동',
    '용강동','대흥동','염리동','신수동','서교동','합정동','망원1동','망원2동','연남동','성산1동','성산2동','상암동','도화동','서강동','공덕동','아현동',
    '목1동','목2동','목3동','목4동','신월1동','신월2동','신월3동','신월4동','신월5동','신월6동','신월7동','신정1동','신정2동','신정3동','신정6동','신정7동','목5동','신정4동',
    '염창동','등촌1동','등촌2동','등촌3동','화곡본동','화곡2동','화곡3동','화곡4동','화곡6동','화곡8동','가양2동','가양3동','발산1동','공항동','방화1동','방화2동','화곡1동','우장산동','가양1동','방화3동',
    '신도림동','구로1동','구로3동','구로4동','구로5동','고척1동','고척2동','개봉2동','개봉3동','오류1동','수궁동','가리봉동','구로2동','개봉1동','오류2동','항동',
    '가산동','독산1동','독산2동','독산3동','독산4동','시흥1동','시흥2동','시흥3동','시흥4동','시흥5동',
    '여의동','당산1동','당산2동','양평1동','양평2동','신길1동','신길3동','신길4동','신길5동','신길6동','신길7동','대림1동','대림2동','대림3동','영등포본동','영등포동','도림동','문래동',
    '노량진2동','상도1동','상도2동','상도3동','상도4동','사당1동','사당3동','사당4동','사당5동','대방동','신대방1동','신대방2동','흑석동','노량진1동','사당2동','보라매동','청림동','행운동','낙성대동','중앙동','인헌동','남현동','서원동','신원동','서림동','신사동','신림동','난향동','조원동','대학동','은천동','성현동','청룡동','난곡동','삼성동','미성동',
    '서초1동','서초2동','서초3동','서초4동','잠원동','반포본동','반포1동','반포2동','반포3동','반포4동','방배본동','방배1동','방배2동','방배3동','방배4동','양재1동','양재2동','내곡동',
    '개포3동','논현1동','논현2동','삼성1동','삼성2동','대치1동','대치4동','역삼1동','역삼2동','도곡1동','도곡2동','개포1동','개포4동','일원본동','일원1동','수서동','세곡동','압구정동','청담동','대치2동','개포2동',
    '풍납1동','풍납2동','거여1동','거여2동','마천1동','마천2동','방이1동','방이2동','오륜동','오금동','송파1동','송파2동','석촌동','삼전동','가락본동','가락1동','가락2동','문정1동','문정2동','잠실본동','잠실4동','잠실6동','잠실7동','잠실2동','잠실3동','장지동','위례동',
    '명일1동','명일2동','고덕1동','고덕2동','암사2동','암사3동','천호1동','천호3동','성내1동','성내2동','성내3동','둔촌1동','둔촌2동','암사1동','천호2동','길동','강일동','상일1동','상일2동'
}

# ===================== 디버그 유틸 =====================

def debug(msg: str):
    if DEBUG:
        print(f'[DEBUG] {msg}')

@contextlib.contextmanager
def stage(name: str):
    t0 = time.time()
    print(f"\n=== [{name}] 시작 ===")
    try:
        yield
    finally:
        dt = time.time() - t0
        print(f"=== [{name}] 종료 ({dt:.2f}s) ===")

In [199]:
# ===================== 경로 & 탐색 유틸 =====================

def resolve_repo_root(start: Path) -> Path:
    for p in [start, *start.parents]:
        if (p / 'DATA').exists() or (p / '.git').exists():
            debug(f'resolve_repo_root -> {p}')
            return p
    debug(f'resolve_repo_root fallback -> {start}')
    return start

def find_shape_auto(repo_root: Path) -> Optional[Path]:
    data_dir = repo_root / 'DATA'
    if not data_dir.exists():
        debug('DATA 디렉터리 없음')
        return None
    matches = list(data_dir.rglob('BND_ADM_DONG_PG*.shp'))
    debug(f'find_shape_auto matches: {len(matches)}')
    return matches[0] if matches else None

def get_shape_path() -> Path:
    if OVERRIDE_SHAPE_PATH:
        if OVERRIDE_SHAPE_PATH.exists():
            print(f'[SHAPE] 사용자 지정 경로 사용: {OVERRIDE_SHAPE_PATH}')
            return OVERRIDE_SHAPE_PATH
        raise FileNotFoundError(f'OVERRIDE_SHAPE_PATH 지정 경로를 찾을 수 없습니다: {OVERRIDE_SHAPE_PATH}')
    repo_root = resolve_repo_root(Path.cwd())
    shp = find_shape_auto(repo_root)
    if shp and shp.exists():
        print(f'[SHAPE] 자동 탐색 성공: {shp}')
        return shp
    raise FileNotFoundError(
        '행정동 Shapefile을 찾지 못했습니다. 수행 옵션:\n'
        '1) <repo_root>/DATA/ 하위에 BND_ADM_DONG_PG* 세트(.shp .shx .dbf .prj) 복사\n'
        '2) OVERRIDE_SHAPE_PATH 에 절대경로 직접 지정 후 재실행\n'
        f'현재 작업 디렉터리: {Path.cwd()}'
    )

# ===================== CSV & 주소 처리 =====================

def load_csv_multi(path: Path, encodings: List[str] | None = None) -> pd.DataFrame:
    with stage('CSV 로드'):
        if encodings is None:
            encodings = ['utf-8-sig','utf-8','cp949','euc-kr']
        last_err = None
        for enc in encodings:
            try:
                df = pd.read_csv(path, encoding=enc)
                print(f'[CSV 로드] {enc} -> {df.shape}')
                debug(f'컬럼: {list(df.columns)[:10]} ...')
                return df
            except UnicodeDecodeError as e:
                last_err = e
                debug(f'인코딩 실패: {enc}')
                continue
        raise RuntimeError(f'인코딩 판별 실패: {last_err}')

def normalize_addr(raw: str) -> Optional[str]:
    if pd.isna(raw) or not str(raw).strip():
        return None
    s = re.sub(r'\s+', ' ', str(raw).strip())
    if not s.startswith(('서울특별시','서울시','서울 ')):
        s = f'{CITY} {s}'
    return s.replace('서울시','서울특별시')

def simplify_addr(addr: str) -> Optional[str]:
    if not addr or pd.isna(addr):
        return None
    s = re.sub(r'\(.*?\)', '', str(addr))
    s = re.sub(r'\s+', ' ', s).strip()
    s = re.split(r'\s\d', s)[0].strip()
    tokens = s.split()
    if len(tokens) >= 3:
        s = ' '.join(tokens[:3])
    return s

# ===================== 지오코딩 =====================

def load_cache(path: Path) -> Dict[str, Tuple[float,float]]:
    with stage('캐시 로드'):
        if path.exists():
            try:
                with open(path,'r',encoding='utf-8') as f:
                    data = json.load(f)
                debug(f'캐시 로드 성공 entries={len(data)}')
                return {k: tuple(v) for k,v in data.items()}
            except Exception as e:
                print('[경고] 캐시 파싱 실패 → 새로 생성', e)
        return {}

def save_cache(cache: Dict[str, Tuple[float,float]], path: Path):
    with stage('캐시 저장'):
        with open(path,'w',encoding='utf-8') as f:
            json.dump(cache, f, ensure_ascii=False, indent=2)
        debug(f'저장 entries={len(cache)} path={path}')

def geocode_primary(df: pd.DataFrame, cache: Dict[str, Tuple[float,float]], delay: float) -> None:
    with stage('지오코딩 1차'):
        geolocator = Nominatim(user_agent=USER_AGENT)
        geocode_func = RateLimiter(geolocator.geocode, min_delay_seconds=delay, swallow_exceptions=True)
        miss = 0
        lat_list, lon_list = [], []
        try:
            from tqdm import tqdm
            iterator = tqdm(df['정규화주소'], desc='Geocoding 1차')
        except ImportError:
            iterator = df['정규화주소']
        for i, addr in enumerate(iterator):
            if pd.isna(addr):
                lat_list.append(None); lon_list.append(None); continue
            if addr in cache:
                lat, lon = cache[addr]
                lat_list.append(lat); lon_list.append(lon); continue
            loc = geocode_func(addr)
            if loc is None:
                lat_list.append(None); lon_list.append(None); miss += 1
            else:
                cache[addr] = (loc.latitude, loc.longitude)
                lat_list.append(loc.latitude); lon_list.append(loc.longitude)
            if DEBUG and (i+1) % PROGRESS_INTERVAL == 0:
                debug(f'진행 {i+1}/{len(df)} 실패누적={miss}')
        df['lat'] = lat_list; df['lon'] = lon_list
        print(f'[지오코딩 1차] 실패 {miss} / 총 {len(df)} (실패율 {(miss/len(df)) if len(df) else 0:.2%})')

def geocode_retry(df: pd.DataFrame, cache: Dict[str, Tuple[float,float]], delay: float) -> None:
    with stage('지오코딩 재시도'):
        remaining = df['lat'].isna().sum()
        if remaining == 0:
            print('[재시도] 실패 없음 → 건너뜀'); return
        geolocator = Nominatim(user_agent=USER_AGENT)
        geocode_func = RateLimiter(geolocator.geocode, min_delay_seconds=delay, swallow_exceptions=True)
        subset = df[df['lat'].isna()].copy(); subset['재시도주소'] = subset['정규화주소'].apply(simplify_addr)
        new_success = 0
        for i, (idx, row) in enumerate(subset.iterrows()):
            addr2 = row['재시도주소']
            if not addr2: continue
            if addr2 in cache:
                lat, lon = cache[addr2]
                df.at[idx,'lat'] = lat; df.at[idx,'lon'] = lon; new_success += 1; continue
            loc = geocode_func(addr2)
            if loc is not None:
                cache[addr2] = (loc.latitude, loc.longitude)
                df.at[idx,'lat'] = loc.latitude; df.at[idx,'lon'] = loc.longitude; new_success += 1
            if DEBUG and (i+1) % 10 == 0:
                debug(f'재시도 진행 {i+1}/{len(subset)} 추가성공={new_success}')
        still = df['lat'].isna().sum()
        print(f'[재시도] 추가 성공 {new_success}, 잔여 실패 {still}')


In [ ]:
# ===================== 경계 / 공간조인 (행정동 기반) =====================

def filter_seoul_by_dong(all_emd: gpd.GeoDataFrame) -> tuple[gpd.GeoDataFrame, str]:
    """전체 행정동 GeoDataFrame에서 서울 행정동만 선택.
    반환: (서울 행정동 subset, 사용된 행정동 컬럼명)
    """
    if all_emd.empty:
        raise ValueError('입력 GeoDataFrame 비어있음')
    lower_map = {c.lower(): c for c in all_emd.columns}
    dong_key_candidates = [
        'adm_dr_nm','adm_nm','emd_kor_nm','emd_nm','행정동','동명','법정동명','법정동'
    ]
    dong_cols = [lower_map[k] for k in dong_key_candidates if k in lower_map]
    if not dong_cols:
        raise KeyError('행정동 후보 컬럼 없음 (filter_seoul_by_dong)')
    best_col = None; best_rate = -1.0; best_series = None
    for c in dong_cols:
        series = all_emd[c].astype(str).str.strip()
        mask = series.isin(SEOUL_DONG_NAMES)
        rate = mask.mean()
        if rate > best_rate:
            best_rate = rate; best_col = c; best_series = series
    if best_rate <= 0:
        raise ValueError('서울 행정동 교집합이 0% 입니다. 리스트/데이터 확인 필요.')
    subset = all_emd[best_series.isin(SEOUL_DONG_NAMES)].copy()
    print(f'[filter] 서울 행정동 {len(subset)} / 전체 {len(all_emd)} (컬럼={best_col}, 매칭률 {best_rate:.2%})')
    return subset, best_col

def load_seoul_boundary(shape_path: Path) -> gpd.GeoDataFrame:
    """Shapefile에서 행정동(법정/행정) 단위 데이터를 불러와
    1) 제공된 서울 행정동 리스트(SEOUL_DONG_NAMES)와 교집합이 가장 큰 '동' 컬럼 탐색
    2) 해당 컬럼으로 서울 데이터만 필터
    3) 자치구(SI/GU) 컬럼을 찾아 dissolve (없으면 EMD_CD 파생)
    """
    with stage('경계 로드/필터(행정동)'):
        all_emd = gpd.read_file(shape_path)
        print('[경계 컬럼]', list(all_emd.columns))
        if all_emd.empty:
            raise ValueError('경계 Shapefile 비어있음')

        lower_map = {c.lower(): c for c in all_emd.columns}
        # 후보 행정동 / 법정동 컬럼 우선순위
        dong_key_candidates = [
            'adm_dr_nm','adm_nm','emd_kor_nm','emd_nm','행정동','동명','법정동명','법정동'
        ]
        dong_cols = [lower_map[k] for k in dong_key_candidates if k in lower_map]
        if not dong_cols:
            raise KeyError('행정동(ADM_DR_NM / ADM_NM 등) 후보 컬럼을 찾지 못했습니다.')

        best_col = None
        best_rate = -1.0
        best_series = None
        for c in dong_cols:
            series = all_emd[c].astype(str).str.strip()
            match_mask = series.isin(SEOUL_DONG_NAMES)
            rate = match_mask.mean()
            print(f'[행정동 매칭] {c}: {match_mask.sum()}/{len(series)} ({rate:.2%})')
            # 가장 높은 매칭률 선택 (동률이면 앞선 순위 유지)
            if rate > best_rate:
                best_rate = rate
                best_col = c
                best_series = series
        if best_rate <= 0:
            raise ValueError('어느 행정동 컬럼도 서울 행정동 리스트와 교집합이 없습니다. 데이터/리스트 검증 필요.')

        print(f'[선택] 행정동 컬럼: {best_col} (매칭률 {best_rate:.2%})')
        mask = best_series.isin(SEOUL_DONG_NAMES)
        seoul_emd = all_emd[mask].copy()
        print(f'[필터] 서울 행정동 추출: {len(seoul_emd)} / {len(all_emd)}')
        if seoul_emd.empty:
            raise ValueError('서울 행정동 필터 결과 0건')

        # 자치구 컬럼 탐색 (이름 우선, 코드 후순위)
        lower_map_seoul = {c.lower(): c for c in seoul_emd.columns}
        sig_name_keys = ['sigungu_nm','sig_kor_nm','sig_nm','시군구명','sig_eng_nm']
        sig_code_keys = ['sig_cd','시군구코드']
        sig_col = None
        for k in sig_name_keys:
            if k in lower_map_seoul:
                sig_col = lower_map_seoul[k]
                print(f'[자치구 식별] 이름 컬럼 사용: {sig_col}')
                break
        if not sig_col:
            for k in sig_code_keys:
                if k in lower_map_seoul:
                    sig_col = lower_map_seoul[k]
                    print(f'[자치구 식별] 코드 컬럼 사용: {sig_col}')
                    break
        # 코드 파생 (EMD_CD 앞 5자리)
        if not sig_col:
            if 'EMD_CD' in seoul_emd.columns:
                seoul_emd['SIG_CD_AUTOGEN'] = seoul_emd['EMD_CD'].astype(str).str.slice(0,5)
                sig_col = 'SIG_CD_AUTOGEN'
                print('[자치구 파생] EMD_CD -> SIG_CD_AUTOGEN 생성')
            else:
                # 다른 코드형 찾기
                for c in seoul_emd.columns:
                    if seoul_emd[c].dtype == object:
                        sample = seoul_emd[c].astype(str).str[:5]
                        if sample.str.isdigit().mean() > 0.9 and sample.str.len().eq(5).mean() > 0.9:
                            seoul_emd['SIG_CD_FALLBACK'] = sample
                            sig_col = 'SIG_CD_FALLBACK'
                            print(f'[자치구 파생] {c} -> SIG_CD_FALLBACK 생성')
                            break
        if not sig_col:
            raise KeyError('자치구 컬럼을 찾거나 생성할 수 없습니다.')

        with stage('경계 Dissolve'):
            seoul_gu = seoul_emd.dissolve(by=sig_col, as_index=False)
            if seoul_gu.crs is None:
                seoul_gu.set_crs(epsg=4326, inplace=True)
            elif seoul_gu.crs.to_string().lower() not in ('epsg:4326','wgs84'):
                seoul_gu = seoul_gu.to_crs(epsg=4326)
        print(f'[경계] 자치구 수 {len(seoul_gu)} (사용 컬럼: {sig_col})')
        return seoul_gu



In [205]:
# ===================== 포인트/조인 =====================

def build_points(df: pd.DataFrame) -> gpd.GeoDataFrame:
    with stage('포인트 생성'):
        pts = df.dropna(subset=['lat','lon']).copy()
        gdf = gpd.GeoDataFrame(pts, geometry=[Point(xy) for xy in zip(pts['lon'], pts['lat'])], crs='EPSG:4326')
        print(f'[포인트] 유효 {len(gdf)} / 전체 {len(df)}')
        return gdf

def spatial_join(pts_gdf: gpd.GeoDataFrame, seoul_gu: gpd.GeoDataFrame) -> tuple[gpd.GeoDataFrame, Optional[str]]:
    with stage('공간조인'):
        priority = ['SIGUNGU_NM','SIG_KOR_NM','SIG_NM','시군구명','SIG_CD','시군구코드','SIG_CD_AUTOGEN','SIG_CD_FALLBACK']
        cand = [c for c in priority if c in seoul_gu.columns]
        admin_col = cand[0] if cand else None
        if admin_col:
            joined = gpd.sjoin(pts_gdf, seoul_gu[[admin_col,'geometry']], how='left', predicate='within')
            match_rate = joined[admin_col].notna().mean()
            print(f'[공간조인] 매칭 비율 {match_rate:.2%} (열: {admin_col})')
            if match_rate < 0.9:
                debug('매칭 비율 90% 미만 → 좌표/경계 CRS 또는 포인트 분포 확인 필요')
            return joined, admin_col
        print('[경고] 자치구 이름 컬럼 없음 → 조인 생략')
        return pts_gdf, None

In [206]:
# === 행정동 컬럼 진단 셀 (SIDO 의존 제거 버전) ===
# 행정동 / 법정동 후보 컬럼의 서울 행정동 매칭률을 출력하여 필터 선택 근거 확인
try:
    shp_path = get_shape_path()
    print('[INFO] Shapefile 경로:', shp_path)
    gdf_tmp = gpd.read_file(shp_path)
    print('[INFO] 전체 컬럼 목록:', list(gdf_tmp.columns))

    lower_map = {c.lower(): c for c in gdf_tmp.columns}
    dong_key_candidates = ['adm_dr_nm','adm_nm','emd_kor_nm','emd_nm','행정동','동명','법정동명','법정동']
    dong_cols = [lower_map[k] for k in dong_key_candidates if k in lower_map]
    if not dong_cols:
        print('[경고] 행정동 후보 컬럼 없음')
    else:
        for c in dong_cols:
            series = gdf_tmp[c].astype(str).str.strip()
            match_mask = series.isin(SEOUL_DONG_NAMES)
            rate = match_mask.mean(); cnt = match_mask.sum()
            print(f'[행정동 후보] {c}: {cnt}/{len(series)} ({rate:.2%})')
            if cnt:
                examples = list(series[match_mask].unique())[:20]
                print('  - 예시:', examples)
except Exception as e:
    print('[ERROR] 행정동 진단 실패:', e)

[DEBUG] resolve_repo_root -> c:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET
[DEBUG] find_shape_auto matches: 1
[SHAPE] 자동 탐색 성공: c:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET\DATA\BND_ADM_DONG_PG\BND_ADM_DONG_PG.shp
[INFO] Shapefile 경로: c:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET\DATA\BND_ADM_DONG_PG\BND_ADM_DONG_PG.shp
[INFO] 전체 컬럼 목록: ['BASE_DATE', 'ADM_CD', 'ADM_NM', 'geometry']
[행정동 후보] ADM_NM: 501/3558 (14.08%)
  - 예시: ['잠실4동', '사직동', '중앙동', '월곡2동', '월곡1동', '잠실7동', '청림동', '서강동', '잠실2동', '성북동', '삼양동', '송정동', '잠실6동', '방이2동', '회현동', '삼청동', '부암동', '평창동', '숭인2동', '용문동']
[INFO] 전체 컬럼 목록: ['BASE_DATE', 'ADM_CD', 'ADM_NM', 'geometry']
[행정동 후보] ADM_NM: 501/3558 (14.08%)
  - 예시: ['잠실4동', '사직동', '중앙동', '월곡2동', '월곡1동', '잠실7동', '청림동', '서강동', '잠실2동', '성북동', '삼양동', '송정동', '잠실6동', '방이2동', '회현동', '삼청동', '부암동', '평창동', '숭인2동', '용문동']


## 🔍 CSV 내 행정동 + 지오코딩 결과 빠른 확인
아래 코드 셀은:
1. 원본 CSV 로드 (다중 인코딩 시도)
2. 주소 컬럼 자동 탐지 → 정규화 주소 생성
3. (lat/lon 없으면) 캐시 기반 지오코딩 수행 (1차 + 재시도)
4. 행정동(있으면) + 주소 + 위경도 샘플 출력

필요시 `SAMPLE_N` 값을 조정해 전체 대신 일부만 지오코딩 테스트할 수 있습니다.

> 지오코딩은 API 호출이므로 처음 한 번 실행 후 캐시에 저장됩니다.

In [207]:
# === CSV 행정동 + 지오코딩 결과 확인 셀 ===
SAMPLE_N = None  # 예: 50 로 설정하면 앞 50개만 지오코딩 테스트
ADDRESS_COL_CANDIDATES = ['대표지번','주소','소재지','지번주소','도로명주소','소재지지번','소재지도로명','위치']
DONG_COL_CANDIDATES = ['행정동','법정동','법정동명','법정동명칭','동','adm_dr_nm','adm_nm','EMD_KOR_NM','EMD_NM']

try:
    df_raw = load_csv_multi(CSV_PATH)
    print('[INFO] 원본 컬럼:', list(df_raw.columns))

    # 행정동 컬럼 탐색(있으면 우선 출력용 보존)
    dong_col = None
    for c in DONG_COL_CANDIDATES:
        if c in df_raw.columns:
            dong_col = c; break
    if dong_col:
        print(f'[행정동 컬럼 감지] {dong_col}')
    else:
        print('[행정동 컬럼] 후보 없음 (Shapefile 기반 행정동 매칭만 가능)')

    # 주소 컬럼 탐색
    addr_col = None
    for c in ADDRESS_COL_CANDIDATES:
        if c in df_raw.columns:
            addr_col = c; break
    if not addr_col:
        raise KeyError('주소 컬럼을 찾지 못했습니다. ADDRESS_COL_CANDIDATES 목록을 수정하세요.')
    print(f'[주소 컬럼 감지] {addr_col}')

    # 주소 정규화
    df_raw['정규화주소'] = df_raw[addr_col].apply(normalize_addr)
    if SAMPLE_N:
        work_df = df_raw.head(SAMPLE_N).copy()
        print(f'[샘플링] {SAMPLE_N}건만 사용')
    else:
        work_df = df_raw.copy()

    # 기존 lat/lon 있으면 재활용
    has_latlon = {'lat','lon'} <= set(work_df.columns)
    cache = load_cache(CACHE_PATH)
    if not has_latlon or work_df['lat'].isna().any() or work_df['lon'].isna().any():
        geocode_primary(work_df, cache, GEOCODE_DELAY_SEC)
        geocode_retry(work_df, cache, GEOCODE_DELAY_SEC)
        save_cache(cache, CACHE_PATH)
    else:
        print('[지오코딩] 기존 lat/lon 컬럼 재사용 (지오코딩 패스)')

    cols_to_show = [c for c in [dong_col, addr_col, '정규화주소', 'lat', 'lon'] if c]
    preview = work_df[cols_to_show].head(30)
    print('\n[미리보기 상위 30]')
    print(preview)

    # 누락 좌표 통계
    miss = work_df['lat'].isna().sum() if 'lat' in work_df.columns else len(work_df)
    print(f"\n[요약] 총 {len(work_df)}건 / 좌표결측 {miss}건 ({(miss/len(work_df)) if len(work_df) else 0:.2%})")
    if miss:
        print('[안내] 결측은 주소 정규화/단순화 규칙 또는 원본 주소 품질 문제일 수 있음')
except Exception as e:
    print('[ERROR] CSV 행정동+지오코딩 확인 중 오류:', e)


=== [CSV 로드] 시작 ===
[CSV 로드] utf-8-sig -> (287, 43)
[DEBUG] 컬럼: ['사업번호', '자치구', '법정동', '사업구분', '운영구분', '추진위원회/조합명', '대표지번', '진행단계', '상태', '토지등 소유자 수'] ...
=== [CSV 로드] 종료 (0.01s) ===
[INFO] 원본 컬럼: ['사업번호', '자치구', '법정동', '사업구분', '운영구분', '추진위원회/조합명', '대표지번', '진행단계', '상태', '토지등 소유자 수', '정비구역명칭', '정비구역위치', '정비구역면적(㎡)', '건축연면적(㎡)', '용도지역', '용도지구', '택지면적(㎡)', '도로면적(㎡)', '공원면적(㎡)', '녹지면적(㎡)', '공공공지면적(㎡)', '학교면적(㎡)', '기타면적(㎡)', '주용도', '건폐율', '용적률', '높이(m)', '지상층수', '지하층수', '분양세대총수', '60㎡이하', '60㎡초과~85㎡이하', '85㎡초과', '임대세대총수', '(임대)40㎡이하', '(임대)40㎡초과~50㎡이하', '(임대)50㎡초과', '건축계획비고', '위치도', '조감도', '배치도', '조합사무실 주소', '조합사무실 전화번호']
[행정동 컬럼 감지] 법정동
[주소 컬럼 감지] 대표지번

=== [캐시 로드] 시작 ===
[DEBUG] 캐시 로드 성공 entries=268
=== [캐시 로드] 종료 (0.01s) ===

=== [지오코딩 1차] 시작 ===
[DEBUG] 진행 200/287 실패누적=21
[DEBUG] 진행 200/287 실패누적=21
[지오코딩 1차] 실패 26 / 총 287 (실패율 9.06%)
=== [지오코딩 1차] 종료 (37.01s) ===

=== [지오코딩 재시도] 시작 ===
[재시도] 추가 성공 26, 잔여 실패 0
=== [지오코딩 재시도] 종료 (0.02s) ===

=== [캐시 저장] 시작 ===
[DEBUG] 저장 entries=268 path

## 🗺 Shapefile 경계 시각화
아래 셀들은:
1. 원본 Shapefile(행정동 단위)를 그대로 요약 및 정적 플롯
2. 행정동 기반 필터 & 자치구 dissolve 후 Folium 인터랙티브 지도 저장
3. GeoJSON export (raw / 자치구)

출력 파일:
- `boundary_raw.html`: 원본(행정동) 경계 미니맵
- `boundary_gu.html`: 자치구 dissolve 결과 인터랙티브 지도
- `boundary_raw.geojson`, `boundary_gu.geojson`: 후속 활용용 GeoJSON

## 🧩 자치구 경계 + 지오코딩 포인트 통합 지도
이 셀은 다음을 수행합니다:
1. 자치구 경계(shapefile → dissolve 결과) 로드
2. CSV 로드 및 (필요시) 지오코딩 캐시 기반 수행
3. 포인트를 Folium 지도에 자치구 경계와 함께 오버레이
4. 포인트 수가 많으면 MarkerCluster 적용 (300초과 시)

출력: `seoul_redev_points_gu_overlay.html` (동일 폴더 저장)

> 실행 전 앞선 함수 정의 셀들을 먼저 실행하세요.

In [209]:
# === 자치구 경계 + 지오코딩 포인트 오버레이 (노트북 즉시 출력) ===
# 전제: 상단에 정의된 함수 (load_csv_multi, normalize_addr, load_cache, geocode_primary, geocode_retry,
#        build_points, load_seoul_boundary, get_shape_path 등) 이미 실행되어 있어야 함.
from IPython.display import display

ADDRESS_COL_CANDIDATES = ['대표지번','주소','소재지','지번주소','도로명주소','소재지지번','소재지도로명','위치']
DONG_COL_CANDIDATES = ['행정동','법정동','법정동명','법정동명칭','동','adm_dr_nm','adm_nm','EMD_KOR_NM','EMD_NM']
OVERLAY_HTML = OUT_DIR / 'seoul_redev_points_gu_overlay.html'

try:
    # 1. 경계 (자치구)
    shp_path = get_shape_path()
    seoul_gu = load_seoul_boundary(shp_path)
    center = [seoul_gu.geometry.centroid.y.mean(), seoul_gu.geometry.centroid.x.mean()]

    # 2. CSV 로드
    df_raw = load_csv_multi(CSV_PATH)
    # 주소 컬럼 탐색
    addr_col = None
    for c in ADDRESS_COL_CANDIDATES:
        if c in df_raw.columns:
            addr_col = c; break
    if not addr_col:
        raise KeyError('주소 컬럼을 찾지 못했습니다. ADDRESS_COL_CANDIDATES 수정 필요')

    # 정규화 주소 생성
    if '정규화주소' not in df_raw.columns:
        df_raw['정규화주소'] = df_raw[addr_col].apply(normalize_addr)

    # 지오코딩 (lat/lon 없거나 결측 존재시) + 캐시
    cache = load_cache(CACHE_PATH)
    need_geocode = ('lat' not in df_raw.columns) or ('lon' not in df_raw.columns) or df_raw['lat'].isna().any() or df_raw['lon'].isna().any()
    if need_geocode:
        if 'lat' not in df_raw.columns: df_raw['lat'] = None
        if 'lon' not in df_raw.columns: df_raw['lon'] = None
        geocode_primary(df_raw, cache, GEOCODE_DELAY_SEC)
        geocode_retry(df_raw, cache, GEOCODE_DELAY_SEC)
        save_cache(cache, CACHE_PATH)
    else:
        print('[지오코딩] 기존 lat/lon 모두 존재 (지오코딩 스킵)')

    # 3. 포인트 GeoDataFrame
    pts_gdf = build_points(df_raw)

    # 4. Folium 지도 생성
    m = folium.Map(location=center, zoom_start=11, tiles='cartodbpositron')

    # (a) 자치구 경계 레이어 (약간 simplify)
    gu_layer = seoul_gu.copy()
    try:
        gu_layer['geometry'] = gu_layer.geometry.simplify(0.0008, preserve_topology=True)
    except Exception:
        pass
    folium.GeoJson(gu_layer.to_json(), name='자치구 경계', style_function=lambda f: {
        'color': '#2c3e50', 'weight': 1, 'fillOpacity': 0.05
    }).add_to(m)

    # (b) 포인트 레이어 (MarkerCluster 조건 적용)
    use_cluster = len(pts_gdf) > 300
    if use_cluster:
        cluster = MarkerCluster(name=f'사업 지점 ({len(pts_gdf)})').add_to(m)
    
    popup_cols = []
    # 행정동/주소 컬럼 후보 중 존재하는 것만 팝업에 포함
    popup_candidates = DONG_COL_CANDIDATES + [addr_col, '정규화주소']
    for pc in popup_candidates:
        if pc in df_raw.columns and pc not in popup_cols:
            popup_cols.append(pc)
    popup_cols = popup_cols[:6]  # 과도한 팝업 길이 제한

    def build_popup(row):
        parts = []
        for c in popup_cols:
            try:
                val = row.get(c, '')
            except Exception:
                val = ''
            if pd.notna(val) and str(val).strip():
                parts.append(f"{c}: {val}")
        return '<br>'.join(parts) if parts else 'N/A'

    for _, r in pts_gdf.iterrows():
        lat, lon = r['lat'], r['lon']
        if pd.isna(lat) or pd.isna(lon):
            continue
        popup_html = build_popup(r)
        marker = folium.CircleMarker(location=[lat, lon], radius=4, color='#e74c3c', fill=True, fill_opacity=0.7,
                                      popup=folium.Popup(popup_html, max_width=300))
        if use_cluster:
            marker.add_to(cluster)
        else:
            marker.add_to(m)

    folium.LayerControl(collapsed=True).add_to(m)

    if SAVE_HTML:
        m.save(OVERLAY_HTML)
        print('[저장] 오버레이 지도 ->', OVERLAY_HTML)
    display(m)
    print(f'[요약] 포인트 {len(pts_gdf)}건, 클러스터 사용={use_cluster}')
    miss_pts = df_raw['lat'].isna().sum()
    if miss_pts:
        print(f'[참고] 좌표 결측 {miss_pts}건 (원본 {len(df_raw)}건)')
except Exception as e:
    print('[ERROR] 오버레이 지도 생성 실패:', e)

[DEBUG] resolve_repo_root -> c:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET
[DEBUG] find_shape_auto matches: 1
[SHAPE] 자동 탐색 성공: c:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET\DATA\BND_ADM_DONG_PG\BND_ADM_DONG_PG.shp

=== [경계 로드/필터(행정동)] 시작 ===
[경계 컬럼] ['BASE_DATE', 'ADM_CD', 'ADM_NM', 'geometry']
[행정동 매칭] ADM_NM: 501/3558 (14.08%)
[선택] 행정동 컬럼: ADM_NM (매칭률 14.08%)
[필터] 서울 행정동 추출: 501 / 3558
[자치구 파생] BASE_DATE -> SIG_CD_FALLBACK 생성

=== [경계 Dissolve] 시작 ===
[경계 컬럼] ['BASE_DATE', 'ADM_CD', 'ADM_NM', 'geometry']
[행정동 매칭] ADM_NM: 501/3558 (14.08%)
[선택] 행정동 컬럼: ADM_NM (매칭률 14.08%)
[필터] 서울 행정동 추출: 501 / 3558
[자치구 파생] BASE_DATE -> SIG_CD_FALLBACK 생성

=== [경계 Dissolve] 시작 ===
=== [경계 Dissolve] 종료 (0.94s) ===
[경계] 자치구 수 1 (사용 컬럼: SIG_CD_FALLBACK)
=== [경계 로드/필터(행정동)] 종료 (1.31s) ===

=== [CSV 로드] 시작 ===
[CSV 로드] utf-8-sig -> (287, 43)
[DEBUG] 컬럼: ['사업번호', '자치구', '법정동', '사업구분', '운영구분', '추진위원회/조합명', '대표지번', '진행단계', '상태', '토지등 소유자 수'] ...
=== [CSV 

C:\Users\Woojin Cho\AppData\Local\Temp\ipykernel_156548\3223068368.py:14: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

  center = [seoul_gu.geometry.centroid.y.mean(), seoul_gu.geometry.centroid.x.mean()]


[DEBUG] 진행 200/287 실패누적=21
[지오코딩 1차] 실패 26 / 총 287 (실패율 9.06%)
=== [지오코딩 1차] 종료 (33.02s) ===

=== [지오코딩 재시도] 시작 ===
[재시도] 추가 성공 26, 잔여 실패 0
=== [지오코딩 재시도] 종료 (0.02s) ===

=== [캐시 저장] 시작 ===
[DEBUG] 저장 entries=268 path=C:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET\outputs\analysis\geocode_cache.json
=== [캐시 저장] 종료 (0.00s) ===

=== [포인트 생성] 시작 ===
[포인트] 유효 287 / 전체 287
=== [포인트 생성] 종료 (0.00s) ===
[지오코딩 1차] 실패 26 / 총 287 (실패율 9.06%)
=== [지오코딩 1차] 종료 (33.02s) ===

=== [지오코딩 재시도] 시작 ===
[재시도] 추가 성공 26, 잔여 실패 0
=== [지오코딩 재시도] 종료 (0.02s) ===

=== [캐시 저장] 시작 ===
[DEBUG] 저장 entries=268 path=C:\Users\Woojin Cho\Documents\GitHub\Urban-Refurbishing-Project_DATA-SET\outputs\analysis\geocode_cache.json
=== [캐시 저장] 종료 (0.00s) ===

=== [포인트 생성] 시작 ===
[포인트] 유효 287 / 전체 287
=== [포인트 생성] 종료 (0.00s) ===


[요약] 포인트 287건, 클러스터 사용=False
